
# Tracking automatique de tempêtes dans les réanalyses ERA5

Calepin à utiliser pour le calcul de trajectoires de tempêtes remarquables à partir de réanalyses ERA5 de Pmer (fichier msl.nc à récupérer sur Copernicus et à mettre dans le dossier "data/storm") :
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form

Les fichiers texte des trajectoires des tempêtes sont stockés dans le répertoire "txt".

In [ ]:
import os

import xarray as xr
import netCDF4

import numpy as np

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

import pandas as pd

from tqdm import tqdm

from scipy.ndimage import maximum_filter, minimum_filter

from shapely.geometry import Point

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

def print_maxmin_points(data, extrema, nsize):
    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        # Print date lon lat and pressure of the minimum
        print(date, data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, int(data[mxy[i], mxx[i]]))

def lonflip(da):
    lon_name = 'longitude'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
def make_boundary_path(lon,lat):
    lons,lats=np.meshgrid(lon,lat)
    boundary_path = np.array([lons[-1,:],lats[-1,:]])
    boundary_path = np.append(boundary_path,np.array([lons[::-1,-1],lats[::-1,-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[1,::-1],lats[1,::-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[:,1],lats[:,1]]),axis=1)
    boundary_path = mpath.Path(np.swapaxes(boundary_path, 0, 1))
    return boundary_path

def make_animation(gif_filepath):
    from PIL import Image
    import os
    from IPython.display import Image as IPImage
    from IPython.display import display
    import time
    
    image_folder = './anim/'+storm+'/' # répertoire contenant les fichiers PNG
    output_file = gif_filepath # nom du fichier de sortie
    animation_speed = 0.9 # vitesse de l'animation en secondes
    
    # Liste tous les fichiers PNG dans le répertoire image_folder
    files = sorted(os.listdir(image_folder))
    image_files = [f for f in files if f.endswith('.png')]
    
    # Ouvre chaque fichier PNG et ajoute l'image à une liste
    images = []
    for filename in image_files:
        img = Image.open(os.path.join(image_folder, filename))
        images.append(img)
    
    # Crée l'animation GIF
    images[0].save(output_file, save_all=True, append_images=images[1:], duration=int(animation_speed*1000), loop=0)
    # Affiche l'animation GIF dans Jupyter
    with open(output_file,'rb') as f:
        display(IPImage(data=f.read(), format='png'))
    # Efface les fichiers PNG
    for filename in image_files:
        os.remove(image_folder+filename)

<div class="alert alert-danger">
<p><b>Réaliser le tracking automatique pour l'ensemble des tempêtes ci-dessous (à l'exception d'Ophélia, Zorbas et Lorenzo) afin de créer les fichiers texte de toutes les trajectoires.</b></p>
</div>

In [ ]:
def print_menu():
    print ('1 -- Tempête de Novembre 1982' )
    print ("2 -- 'Ourangan' d'octobre 1987")
    print ('3 -- Herta - Février 1990' )
    print ('4 -- Viviane - Février 1990' )
    print ('5 -- Braer - Janvier 1993' )
    print ('6 -- Lothar - Décembre 1999' )
    print ('7 -- Martin - Décembre 1999' )
    print ('8 -- Klaus - Janvier 2009' )
    print ('9 -- Xynthia - Février 2010' )
    print ('10 -- Joachim - Décembre 2011' )
    print ('11 -- Zeus - Mars 2017' )
    print ('12 -- Ophelia - Octobre 2017 (transition extra tropicale)' )
    print ('13 -- Eleanor - Janvier 2018' )
    print ('14 -- Zorbas - Septembre 2018 (medicane)')
    print ('15 -- Lorenzo - Octobre 2019 (transition extra tropicale)' )
    print ('16 -- Alex - Octobre 2020' )
    print ('17 -- Ciaran - Octobre-novembre 2023' )

print_menu()

option = int(input('Enter number of the desired storm : ')) 
if option == 1:
    storm='Nov1982'
elif option == 2:
    storm='Oct1987'
elif option == 3:
    storm='Herta'
elif option == 4:
    storm='Viviane'
elif option == 5:
    storm='Braer'
elif option == 6:
    storm='Lothar'
elif option == 7:
    storm='Martin'
elif option == 8:
    storm='Klaus'
elif option == 9:
    storm='Xynthia'
elif option == 10:
    storm='Joachim'
elif option == 11:
    storm='Zeus'
elif option == 12:
    storm='Ophelia'
elif option == 13:
    storm='Eleanor'
elif option == 14:
    storm='Zorbas'
elif option == 15:
    storm='Lorenzo'
elif option == 16:
    storm='Alex'
elif option == 17:
    storm='Ciaran'

else:
    print('Invalid option. Please enter a number between 1 and 17.')
    
if not os.path.exists('./anim/'+storm):
    os.mkdir('./anim/'+storm)

if not os.path.exists('./figs/'+storm):
    os.mkdir('./figs/'+storm)
    
dir_anim ='./anim/'+storm+'/'
dir_data ='./data/'+storm+'/'

if os.path.exists(dir_data+"msl.nc") or os.path.exists(dir_data+"msl1.nc"):
    print('All good : MSLP file is present in the folder data/'+storm)
else:
    print('Warning : MSLP file is not present in the folder data/'+storm)

In [ ]:
if storm=='Nov1982':
    date1='1982-11-06T03'
    date2='1982-11-08T12'                                 
if storm=='Oct1987':
    date1='1987-10-15T09'
    date2='1987-10-16T23'                               
if storm=='Herta':
    date1='1990-02-02T09'
    date2='1990-02-04T05'                                 
if storm=='Viviane':
    date1='1990-02-26T00'
    date2='1990-02-28T00'
if storm=='Braer':
    date1='1993-01-08T22'
    date2='1993-01-13T23'
if storm=='Lothar':
    date1='1999-12-25T00'
    date2='1999-12-26T15'                                 
if storm=='Martin':
    date1='1999-12-26T12'
    date2='1999-12-28T04'                                 
if storm=='Klaus':
    date1='2009-01-23T06'
    date2='2009-01-24T15'                                 
if storm=='Xynthia':
    date1='2010-02-26T21'
    date2='2010-02-28T21'                                 
if storm=='Joachim':
    date1='2011-12-15T03'
    date2='2011-12-17T23'
if storm=='Zeus':
    date1='2017-03-06T04'
    date2='2017-03-07T23' 
if storm=='Ophelia':
    date1='2017-10-14T00'
    date2='2017-10-17T21'                                
if storm=='Eleanor':
    date1='2018-01-02T20'
    date2='2018-01-04T10'
if storm=='Zorbas':
    date1='2018-09-27T05'
    date2='2018-09-30T23'
if storm=='Lorenzo':
    date1='2019-10-01T00'
    date2='2019-10-04T23'  
if storm=='Alex':
    date1='2020-10-01T14'
    date2='2020-10-03T06'
if storm=='Ciaran':
    date1='2023-10-30T18'
    date2='2023-11-03T22'

In [ ]:
latS=30
latN=70
lonW=-60
lonE=30

if storm=='Braer':
    latS=40
    latN=80
    lonW=-80
    lonE=20
if storm=='Ciaran':
    latS=30
    latN=70
    lonW=-70
    lonE=30
if storm=='Lorenzo':
    latS=25
    latN=60
    lonW=-60
    lonE=20
if storm=='Zorbas':
    latS=30
    latN=45
    lonW=5
    lonE=35
    
f1    = xr.open_mfdataset(dir_data+"msl*.nc").sel(time=slice(date1,date2)).sel(latitude=slice(latN,latS))
print(f1)

mslp0 = f1['msl']/100
lat  = mslp0.latitude.values
time  = mslp0.time.values

mslp = lonflip(mslp0)
mslp=mslp.sel(longitude=slice(lonW,lonE))

lon  = mslp.longitude.values

print(mslp)

In [ ]:
%%capture cap --no-stderr
for i in tqdm(range(len(time))):
    date=str(time[i])[0:13]   
    print_maxmin_points(mslp[i,:,:], 'min', 25)

In [ ]:
file_era = './txt/era5_minimums_'+date1+'_'+date2+'.txt'
with open(file_era, 'w') as f:
    f.write(cap.stdout)
f.close()

In [ ]:
buffer=3
if storm=='Martin':
    buffer=4
if storm=='Braer':
    buffer=4
    
def tracking(file): 
    df = pd.read_csv(file,sep=" ",header=None)
    liste_date =  np.unique(df[0].values)
    ds = df.to_xarray()
    
    # We will track lows that are present at initial time
    original_position = ds.sel(index= ds[0] == liste_date[0])
    
    # Build individual tracking for each detected low
    traj = []
    for ind in original_position.index.values: # Boucle sur les depressions à t=0
        position = Point(original_position.sel(index=ind)[1],original_position.sel(index=ind)[2])
        dep_traj = []
        dep_traj.append((
            original_position.sel(index=ind)[1].values,
            original_position.sel(index=ind)[2].values,
            original_position.sel(index=ind)[0].values,
            original_position.sel(index=ind)[3].values, 
            ))
        for date in liste_date[1:]: # Loop for all timesteps
            temp = ds.sel(index= ds[0] == date)
            l_area = []
            for other_idx in temp.index.values: 
                oth_pos = Point(temp.sel(index=other_idx)[1],temp.sel(index=other_idx)[2])
                l_area.append(position.buffer(buffer).intersection(oth_pos.buffer(buffer)).area) 
                # Finding intersection between two circles of 1°. 
            if np.max(l_area)>0.001: # Compare areas
                elt = np.argmax(l_area)
                n_position=Point(temp.isel(index=elt)[1],temp.isel(index=elt)[2])
                dep_traj.append([
                    temp.isel(index=elt)[1].values,
                    temp.isel(index=elt)[2].values,
                    temp.isel(index=elt)[0].values,
                    temp.isel(index=elt)[3].values] )
                position = n_position
            else: 
                break
        traj.append(dep_traj)
    return traj

In [ ]:
tracking_era = tracking(file_era)
print("Number of tracked lows starting at "+date1+": ", len(tracking_era))

In [ ]:
print(tracking_era[0][0])
print(tracking_era[1][0])

In [ ]:
def get_list(track): 
    list_lat = []
    list_lon = []
    list_time = []
    list_pres = []
    for i in range(len(track)):
        list_lon.append(track[i][0])
        list_lat.append(track[i][1])
        list_time.append(track[i][2])
        list_pres.append(track[i][3])
    return list_lon, list_lat, list_time, list_pres

In [ ]:
file_storms = './txt/era5_tracks_'+date1+'.txt'
file=open(file_storms, "w+")

for i in range(len(tracking_era)):
    file.write("#")
    #file.write("\n")

    liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[i])
    list_time = [str(x) for x in liste_time]
    
    for j in range(len(tracking_era[i])):
        file.write(str(liste_time[j]))
        file.write(' ')
        file.write(str(liste_lon[j]))
        file.write(' ')
        file.write(str(liste_lat[j]))
        file.write(' ')
        file.write(str(liste_pres[j]))
        file.write("\n")
file.close()

In [ ]:
mslp_levels = np.arange(900,1072,2)
projection=ccrs.NearsidePerspective(central_longitude=(lonW+lonE)/2, central_latitude=(latS+latN)/2)
if storm=='Zorbas':
    projection=ccrs.PlateCarree()

bounds = [(lonW, lonE, latS, latN)]

fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(111, projection=projection)
ax.set_title('Tracks of all systems detected at '+date1,loc='center',fontsize=14)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
LAND = cfeature.NaturalEarthFeature('physical', 'land', '10m',edgecolor='face',facecolor=cfeature.COLORS['land'],linewidth=.1)
ax.add_feature(LAND)
OCEAN = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',edgecolor='face',facecolor=cfeature.COLORS['water'],linewidth=.1)
ax.add_feature(OCEAN)
ax.gridlines(draw_labels=False, color='gray', alpha=0.8, linestyle='-')
c = ax.contour(lon, lat, mslp[0,:,:], levels=mslp_levels, colors="grey", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c,fmt='%4.1i',fontsize=10)

for ind in range(len(tracking_era)):
    liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[ind])
    ax.plot(liste_lon,liste_lat, label=ind, transform=ccrs.PlateCarree())
    ax.scatter(liste_lon[0],liste_lat[0], color='green', transform=ccrs.PlateCarree())
    ax.scatter(liste_lon[-1],liste_lat[-1], color='red', transform=ccrs.PlateCarree())
    ax.text(liste_lon[0], liste_lat[0], liste_pres[0],verticalalignment='top', horizontalalignment='center',
            transform=ccrs.PlateCarree())
    ax.text(liste_lon[-1], liste_lat[-1], liste_pres[-1],verticalalignment='top', horizontalalignment='center',
            transform=ccrs.PlateCarree())
plt.legend(loc='right')
plt.show()

figname='./figs/'+storm+'/tracks_'+date1
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(111, projection=projection)
ax.set_title('Tracks of all systems detected at '+date1+' and lasting at least 24h',loc='center',fontsize=14)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
boundary_path = make_boundary_path(lon, lat)
ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
LAND = cfeature.NaturalEarthFeature('physical', 'land', '10m',edgecolor='face',facecolor=cfeature.COLORS['land'],linewidth=.1)
ax.add_feature(LAND)
OCEAN = cfeature.NaturalEarthFeature('physical', 'ocean', '10m',edgecolor='face',facecolor=cfeature.COLORS['water'],linewidth=.1)
ax.add_feature(OCEAN)
ax.gridlines(draw_labels=False, color='gray', alpha=0.8, linestyle='-')
c = ax.contour(lon, lat, mslp[0,:,:], levels=mslp_levels, colors="grey", linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c,fmt='%4.1i',fontsize=10)

for ind in range(len(tracking_era)):
    liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[ind])
    if len(liste_lon) >24: 
        ax.plot(liste_lon,liste_lat, label=ind, transform=ccrs.PlateCarree())
        ax.scatter(liste_lon[0],liste_lat[0], color='green', transform=ccrs.PlateCarree())
        ax.scatter(liste_lon[-1],liste_lat[-1], color='red', transform=ccrs.PlateCarree())
        ax.text(liste_lon[0], liste_lat[0], liste_pres[0],verticalalignment='top', horizontalalignment='center',
                transform=ccrs.PlateCarree())
        ax.text(liste_lon[-1], liste_lat[-1], liste_pres[-1],verticalalignment='top', horizontalalignment='center',
                transform=ccrs.PlateCarree())
plt.legend(loc='right')
plt.show()

figname='./figs/'+storm+'/tracks24h_'+date1
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
ind_era = int(input("Enter index of storm "+storm+" : "))
print('Number of points for the desired low : '+str(len(tracking_era[ind_era])))

In [ ]:
### Méthode 1 : A partir du fichier texte des trajectoires (grâce au séparateur #)

with open(file_storms, 'r', encoding='utf-8') as file:
    contents = file.read()
    character = '#'
    result = contents.split(character, len(tracking_era))
print(result[ind_era+1])

file_storm = './txt/'+storm+'.txt'
with open(file_storm, 'w') as f:
    f.write(result[ind_era+1])
    
### Méthode 2 : avec la fonction get_list()

#liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[ind_era])

#file_storm = './txt/'+storm+'.txt'
#file=open(file_storm, "w+")
#for i in range(len(tracking_era[ind_era])):
#    file.write(str(liste_time[i]))
#    file.write(' ')
#    file.write(str(liste_lon[i]))
#    file.write(' ')
#    file.write(str(liste_lat[i]))
#    file.write(' ')
#    file.write(str(liste_pres[i]))
#    file.write("\n")
#file.close()

In [ ]:
liste_lon, liste_lat, liste_time, liste_pres = get_list(tracking_era[ind_era])
list_time = [str(x) for x in liste_time]

for i in tqdm(range(len(tracking_era[ind_era]))):
    #print(str(tracking_era[ind_era][i][2])[0:13])
    fig = plt.figure(figsize=(15., 10.))

    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Storm '+storm+' - MSLP and tracking : '+list_time[0]+' to ' +list_time[-1],loc='left',fontsize=14)
    ax.set_title(str(time[i])[0:13],loc='right',fontsize=14)
    ax.coastlines("10m", color='grey', zorder=3)
    ax.gridlines(draw_labels=False, color='gray', alpha=0.8, linestyle='-')
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())  
    
    c1 = ax.contour(lon, lat, mslp[i,:,:], levels=mslp_levels, colors="black", linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c1,fmt='%4.1i',fontsize=10)
    plot_maxmin_points(mslp[i,:,:], 'min', 25,
                       symbol='L', color='b', transform=ccrs.PlateCarree()) 

    ax.plot(liste_lon[0:i+1],liste_lat[0:i+1], c='red', marker='+', transform=ccrs.PlateCarree())
    
    figname='./anim/'+storm+'/MSL_tracking2_'+list_time[i]
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
gif_filepath = './anim/'+storm+'/MSL_tracking2.gif'
make_animation(gif_filepath)